In [44]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()


In [45]:
from sklearn.datasets import load_boston

boston = load_boston()
#print(boston)

In [46]:
print(boston.data.shape)
print(len(boston.target))
print(boston.feature_names)

(506, 13)
506
['CRIM' 'ZN' 'INDUS' 'CHAS' 'NOX' 'RM' 'AGE' 'DIS' 'RAD' 'TAX' 'PTRATIO'
 'B' 'LSTAT']


In [47]:
import numpy as np
X= np.array(boston.data)
y= np.array(boston.target)


In [48]:
num_training_sample = X.shape[0]
num_features = X.shape[1]



In [49]:
def normalize(dataset):
    mean = np.mean(dataset,axis=0)
    std = np.std(dataset,axis=0)
    return (dataset-mean )/std 

In [50]:
X_normalized = normalize(X)
print(X)


[[6.3200e-03 1.8000e+01 2.3100e+00 ... 1.5300e+01 3.9690e+02 4.9800e+00]
 [2.7310e-02 0.0000e+00 7.0700e+00 ... 1.7800e+01 3.9690e+02 9.1400e+00]
 [2.7290e-02 0.0000e+00 7.0700e+00 ... 1.7800e+01 3.9283e+02 4.0300e+00]
 ...
 [6.0760e-02 0.0000e+00 1.1930e+01 ... 2.1000e+01 3.9690e+02 5.6400e+00]
 [1.0959e-01 0.0000e+00 1.1930e+01 ... 2.1000e+01 3.9345e+02 6.4800e+00]
 [4.7410e-02 0.0000e+00 1.1930e+01 ... 2.1000e+01 3.9690e+02 7.8800e+00]]


In [51]:
print(X_normalized)

[[-0.41978194  0.28482986 -1.2879095  ... -1.45900038  0.44105193
  -1.0755623 ]
 [-0.41733926 -0.48772236 -0.59338101 ... -0.30309415  0.44105193
  -0.49243937]
 [-0.41734159 -0.48772236 -0.59338101 ... -0.30309415  0.39642699
  -1.2087274 ]
 ...
 [-0.41344658 -0.48772236  0.11573841 ...  1.17646583  0.44105193
  -0.98304761]
 [-0.40776407 -0.48772236  0.11573841 ...  1.17646583  0.4032249
  -0.86530163]
 [-0.41500016 -0.48772236  0.11573841 ...  1.17646583  0.44105193
  -0.66905833]]


In [52]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X_normalized,y,random_state=42,test_size =0.33)

print(X_train.shape)
print(X_test.shape)

(339, 13)
(167, 13)


In [53]:
X_train = np.transpose(X_train)
print(X_train.shape)

(13, 339)


In [54]:
X_test = np.transpose(X_test)
print(X_test.shape)

(13, 167)


In [55]:
print(y_train.shape)
print(y_test.shape)

(339,)
(167,)


In [56]:
y_train = np.reshape(y_train,(1,len(y_train)))
y_test = np.reshape(y_test,(1,len(y_test)))
print(y_train.shape)
print(y_test.shape)

(1, 339)
(1, 167)


In [57]:
tf.reset_default_graph()
X_tf = tf.placeholder(tf.float32,[num_features,None])
y_tf = tf.placeholder(tf.float32,[1,None])

learning_rate = tf.placeholder(tf.float32,shape=())
W = tf.Variable(tf.ones([num_features,1]))
b = tf.Variable(tf.zeros(1))

init = tf.global_variables_initializer()
W_t = tf.transpose(W)
W_t_X = tf.matmul(W_t,X_tf)
y_pred = W_t_X + b
cost = tf.reduce_mean(tf.square(y_tf - y_pred))

training_step = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(cost)

In [58]:
def linear_model(learning_r,training_epochs,train_data,train_labels,debug=False):
    sess = tf.Session()
    sess.run(init)
    cost_history = np.empty(shape=[0],dtype='float')
    feed_dict = {X_tf:train_data,y_tf:train_labels,learning_rate:learning_r}
    for epoch in range(training_epochs+1):
        cost_val = sess.run(cost,feed_dict)
        cost_history = np.append(cost_history,cost_val)
        sess.run(training_step,feed_dict)
        if epoch % 10 == 0 & debug:
            print('Reached epoch ',epoch,' cost =',str.format('{0:.6f}',cost_val))
            
    return sess,cost_history
  

In [59]:
sess,cost = linear_model(learning_r=0.01,training_epochs=1920,train_data=X_train,train_labels=y_train,debug=True)

Reached epoch  0  cost = 668.342651
Reached epoch  10  cost = 392.752319
Reached epoch  20  cost = 262.624847
Reached epoch  30  cost = 181.894653
Reached epoch  40  cost = 129.436020
Reached epoch  50  cost = 94.946548
Reached epoch  60  cost = 72.109024
Reached epoch  70  cost = 56.890278
Reached epoch  80  cost = 46.684696
Reached epoch  90  cost = 39.796509
Reached epoch  100  cost = 35.114952
Reached epoch  110  cost = 31.908476
Reached epoch  120  cost = 29.692829
Reached epoch  130  cost = 28.146111
Reached epoch  140  cost = 27.053362
Reached epoch  150  cost = 26.270508
Reached epoch  160  cost = 25.700592
Reached epoch  170  cost = 25.278101
Reached epoch  180  cost = 24.958595
Reached epoch  190  cost = 24.711782
Reached epoch  200  cost = 24.516901
Reached epoch  210  cost = 24.359657
Reached epoch  220  cost = 24.230106
Reached epoch  230  cost = 24.121311
Reached epoch  240  cost = 24.028358
Reached epoch  250  cost = 23.947750
Reached epoch  260  cost = 23.876949
Reached

In [60]:
#Another construction node and session run for calculating accuracy.

accuracy = tf.keras.losses.MSE(y_test,y_pred)

sess.run(accuracy,feed_dict={X_tf:X_test})
 


array([20.811172], dtype=float32)

In [61]:
sess.close()